In [152]:
import six.moves.cPickle as pickle
import numpy as np
import string
import re
import random
import math
import pandas as pd
from collections import Counter
from itertools import *
import tqdm

In [301]:
relation_f = ["l_embedding.txt","f_embedding.txt", "i_embedding.txt","c_embedding.txt"]

#store relational data 
for i in range(len(relation_f)):
    f_name = relation_f[i]
    max_id = 0
    with open('../data/2007_1_1/'+f_name, "r") as neigh_f:
        for line in islice(neigh_f, 1, None):
            values = line.split()
            index = int(values[0])
            if index>max_id:
                max_id = index
    print(f_name[0], max_id+1)

l 22309
f 6498
i 90024
c 45906


In [209]:
# 构建节点的一阶邻居
l_f_list_train = [[] for k in range(L_n)]
f_l_list_train = [[] for k in range(F_n)]
f_i_list_train = [[] for k in range(F_n)] 
i_f_list_train = [[] for k in range(I_n)] 
i_c_list_train = [[] for k in range(I_n)]
c_i_list_train = [[] for k in range(C_n)]
c_c_list_train = [[] for k in range(C_n)]

relation_f = ["l_f_list_train.txt", "f_l_list_train.txt",\
		 "f_i_list_train.txt", "i_f_list_train.txt",\
		 "i_c_list_train.txt","c_i_list_train.txt",\
		 "c_c_list_train.txt"]

#store relational data 
for i in range(len(relation_f)):
    f_name = relation_f[i]
    with open('../data/2007_1_1/'+f_name, "r") as neigh_f:
        for line in neigh_f:
            line = line.strip()
            node_id = int(re.split(':', line)[0])
            neigh_list = re.split(':', line)[1]
            # if node_id == 29956:
            #     print(neigh_list)
            neigh_list_id = re.split(',', neigh_list)
            for j in range(len(neigh_list_id)):
                rela_list_train = eval(f_name[:-4])
                # rela_list_train[node_id].append(f_name[2]+str(neigh_list_id[j]))
                list_train_set = set(rela_list_train[node_id])
                list_train_set.add(f_name[2]+str(neigh_list_id[j]))
                rela_list_train[node_id] = list(list_train_set)

In [210]:
l_neigh_list_train = [[] for k in range(L_n)]
for i in range(L_n):
    l_neigh_list_train[i] += l_f_list_train[i]
    
f_neigh_list_train = [[] for k in range(F_n)]
for i in range(F_n):
    f_neigh_list_train[i] += f_l_list_train[i]
    f_neigh_list_train[i] += f_i_list_train[i] 
    
i_neigh_list_train = [[] for k in range(I_n)]
for i in range(I_n):
    i_neigh_list_train[i] += i_f_list_train[i]
    i_neigh_list_train[i] += i_c_list_train[i] 
    
c_neigh_list_train = [[] for k in range(C_n)]
for i in range(C_n):
    c_neigh_list_train[i] += c_i_list_train[i]
    c_neigh_list_train[i] += c_c_list_train[i] 

In [271]:
node_type = ['l','f','i','c']
list_train_list = [l_f_list_train, f_neigh_list_train, i_neigh_list_train, c_neigh_list_train]
walk_n = 10
walk_L= 30
data_path = '../data/2007_1_1/'
def gen_het_rand_walk():
	het_walk_f = open(data_path + "het_random_walk.txt", "w")
	#print len(p_neigh_list_train)
	for i in range(walk_n):
		for j in range(L_n):
			if len(l_f_list_train[j]):
				curNode = "l" + str(j)
				het_walk_f.write(curNode + " ")
				for l in range(walk_L - 1):
					node_idx = node_type.index(curNode[0])
					cur_idx = int(curNode[1:])
					# print(cur_idx)
					curNode = random.choice(list_train_list[node_idx][cur_idx])
					# print(node_idx, curNode)
					het_walk_f.write(curNode + " ")
				het_walk_f.write("\n")
	het_walk_f.close()

In [159]:
gen_het_rand_walk()

In [160]:
# import string
# import re
# import random
# import math
# import numpy as np
from gensim.models import Word2Vec
from itertools import *
dimen = 384
window = 5


def read_random_walk_corpus():
	walks=[]
	inputfile = open("../data/2007_1_1/het_random_walk.txt", "r")
	for line in inputfile:
		path = []
		node_list=re.split(' ',line)
		for i in range(len(node_list)):
			path.append(node_list[i])
		walks.append(path)
	inputfile.close()
	return walks


walk_corpus = read_random_walk_corpus()
model = Word2Vec(walk_corpus, vector_size = dimen, window = window, min_count = 0, workers = 2, sg = 1, hs = 0, negative = 5)


print("Output...")
#model.wv.save_word2vec_format("../data/node_embedding.txt")
model.wv.save_word2vec_format("../data/2007_1_1/node_net_embedding.txt")

Output...
